# Experiment Analysis Framework

This notebook aggregates prior training artifacts from **neural-network-lab-python**, surfaces diagnostic visualizations, and recommends data-driven hyperparameter refinements for future experiments. It is designed to be reusable across training runs with minimal manual setup.

## Workflow Overview

1. Validate the presence of required configs, logs, scalers, and weight checkpoints.
2. Load active and historical configuration payloads and align them with training outcomes.
3. Ingest `loss_history.csv`, `training_results.csv`, and particle simulation data for analytics.
4. Reconstruct the latest model checkpoint, generate predictions, and evaluate residuals.
5. Render visual diagnostics (loss curves, learning-rate sweeps, residual histograms, correlation heatmap).
6. Summarize run health, recommend hyperparameter sweeps, and capture actionable next steps.

In [ ]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Sequence, Tuple

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from IPython.display import Markdown, display

from advanced_neural_network import AdvancedNeuralNetwork
from data_processing import complete_data_pipeline, load_and_validate_data
from ml_utils import compute_loss_weights
from weight_constraints import BinaryWeightConstraintChanges, BinaryWeightConstraintMax, OscillationDampener

pd.options.display.max_rows = 60
pd.options.display.float_format = '{:,.4f}'.format

sns.set_theme(style="whitegrid")

In [ ]:
PROJECT_NAME = "neural-network-lab-python"

INPUT_FEATURES = [
    "mass",
    "initial_velocity_x",
    "initial_velocity_y",
    "initial_position_x",
    "initial_position_y",
    "charge",
    "magnetic_field_strength",
    "simulation_time"
]

OUTPUT_TARGETS = [
    "final_velocity_x",
    "final_velocity_y",
    "final_position_x",
    "final_position_y",
    "kinetic_energy",
    "trajectory_length"
]

ANALYSIS_SEED = 42

np.random.seed(ANALYSIS_SEED)
tf.random.set_seed(ANALYSIS_SEED)


def format_bytes(size: Optional[int]) -> Optional[str]:
    """Format raw byte counts into human readable text."""
    if size is None: return None

    threshold = 1024.0

    units = ("B", "KB", "MB", "GB", "TB")

    value = float(size)

    for unit in units:
        if value < threshold or unit == units[-1]: return f"{value:.1f} {unit}"

        value /= threshold


def resolve_project_paths() -> Dict[str, Path]:
    """Resolve key project directories relative to this notebook."""
    root = Path.cwd()

    if root.name != PROJECT_NAME:
        for parent in root.parents:
            if parent.name == PROJECT_NAME: root = parent

    config_dir = root / "ml_config"

    output_dir = root / "training_output"

    analysis_dir = output_dir / "analysis"

    figures_dir = analysis_dir / "figures"

    analysis_dir.mkdir(parents=True, exist_ok=True)

    figures_dir.mkdir(parents=True, exist_ok=True)

    return {
        "project_root": root,
        "config_dir": config_dir,
        "output_dir": output_dir,
        "analysis_dir": analysis_dir,
        "figures_dir": figures_dir,
        "data_path": root / "particle_data.csv",
        "scaler_X": root / "scaler_X.pkl",
        "scaler_y": root / "scaler_y.pkl",
        "weight_dir": root / "saved_weights/"
    }


def validate_required_artifacts(paths: Dict[str, Path]) -> pd.DataFrame:
    """Check presence and metadata of required artifacts."""
    required = {
        "model_config": paths["config_dir"] / "model_config.json",
        "training_config": paths["config_dir"] / "training_config.json",
        "loss_history": paths["output_dir"] / "loss_history.csv",
        "training_results": paths["output_dir"] / "training_results.csv",
        "configuration_log": paths["output_dir"] / "configuration_log.csv",
        "particle_data": paths["data_path"],
        "scaler_X": paths["scaler_X"],
        "scaler_y": paths["scaler_y"]
    }

    optional = {
        "analysis_dir": paths["analysis_dir"],
        "figures_dir": paths["figures_dir"]
    }

    notes = {
        "particle_data": "Regenerate via data pipeline if missing.",
        "scaler_X": "Rebuilt automatically through complete_data_pipeline.",
        "scaler_y": "Rebuilt automatically through complete_data_pipeline."
    }

    records: List[Dict[str, Any]] = []

    def append_record(label: str, path: Path, critical: bool) -> None:
        exists = path.exists()

        size = path.stat().st_size if exists and path.is_file() else None

        modified = pd.Timestamp(path.stat().st_mtime, unit="s") if exists else None

        records.append({
            "artifact": label,
            "critical": critical,
            "exists": exists,
            "path": str(path.relative_to(paths["project_root"])) if exists else str(path),
            "size_bytes": size,
            "size_readable": format_bytes(size),
            "modified": modified,
            "note": notes.get(label)
        })

    for label, path in required.items():
        append_record(label, path, True)

    for label, path in optional.items():
        append_record(label, path, False)

    status_df = pd.DataFrame(records)

    if status_df.empty: return status_df

    status_df = status_df.sort_values(["critical", "artifact"], ascending=[False, True]).reset_index(drop=True)

    return status_df


def list_checkpoint_weights(paths: Dict[str, Path]) -> pd.DataFrame:
    """List available weight checkpoints with epoch metadata."""
    pattern = "model_weights_epoch_*.weights.h5"

    checkpoint_files = sorted(paths["weight_dir"].glob(pattern))

    rows: List[Dict[str, Any]] = []

    for file_path in checkpoint_files:
        name = file_path.name

        parts = name.split("_")

        epoch_token = parts[3] if len(parts) > 3 else parts[-1]

        epoch = int(epoch_token.replace(".weights.h5", "")) if epoch_token else None

        rows.append({
            "epoch": epoch,
            "name": name,
            "path": str(file_path.relative_to(paths["project_root"])) if file_path.exists() else str(file_path),
            "modified": pd.Timestamp(file_path.stat().st_mtime, unit="s"),
            "size_bytes": file_path.stat().st_size
        })

    checkpoint_df = pd.DataFrame(rows)

    if checkpoint_df.empty: return checkpoint_df

    checkpoint_df = checkpoint_df.sort_values("epoch").reset_index(drop=True)

    latest_epoch = checkpoint_df["epoch"].max()

    checkpoint_df["size_readable"] = checkpoint_df["size_bytes"].apply(format_bytes)

    checkpoint_df["is_latest"] = checkpoint_df["epoch"] == latest_epoch

    return checkpoint_df

In [ ]:
def load_configs(paths: Dict[str, Path]) -> Tuple[Dict[str, Any], Dict[str, Any], pd.DataFrame]:
    """Load active configs and historical configuration snapshots with derived metrics."""
    model_config_path = paths["config_dir"] / "model_config.json"

    training_config_path = paths["config_dir"] / "training_config.json"

    with model_config_path.open() as handle:
        model_config = json.load(handle)

    with training_config_path.open() as handle:
        training_config = json.load(handle)

    snapshots: List[Dict[str, Any]] = []

    for config_path in sorted(paths["output_dir"].glob("training_config_*.json")):
        with config_path.open() as handle:
            payload = json.load(handle)

        combined: Dict[str, Any] = {
            "config_id": payload.get("config_id"),
            "timestamp": payload.get("timestamp")
        }

        model_payload = payload.get("model_config", {})

        for key, value in model_payload.items():
            combined[key] = value

        training_payload = payload.get("training_config", {})

        for key, value in training_payload.items():
            combined[f"train_{key}"] = value

        summary_payload = payload.get("performance_summary", {})

        combined["best_r2"] = summary_payload.get("best_r2")
        combined["final_r2"] = summary_payload.get("current_r2")
        combined["best_epoch"] = summary_payload.get("best_r2_epoch")
        combined["avg_epoch_time_logged"] = summary_payload.get("avg_epoch_time")
        combined["total_training_time"] = summary_payload.get("total_training_time")
        combined["weight_modifications_used"] = summary_payload.get("weight_modifications_used")
        combined["peak_memory_mb"] = summary_payload.get("peak_memory_mb")

        snapshots.append(combined)

    snapshots_df = pd.DataFrame(snapshots)

    if snapshots_df.empty: return model_config, training_config, snapshots_df

    snapshots_df["timestamp"] = pd.to_datetime(snapshots_df["timestamp"])

    if {"total_training_time", "train_epochs"}.issubset(snapshots_df.columns):
        snapshots_df["avg_epoch_time_calc"] = snapshots_df["total_training_time"] / snapshots_df["train_epochs"]

    snapshots_df["r2_delta"] = snapshots_df["best_r2"] - snapshots_df["final_r2"]

    snapshots_df = snapshots_df.sort_values("timestamp").reset_index(drop=True)

    return model_config, training_config, snapshots_df


def load_training_logs(paths: Dict[str, Path]) -> Dict[str, pd.DataFrame]:
    """Load loss history and training results with derived analytics."""
    loss_path = paths["output_dir"] / "loss_history.csv"

    results_path = paths["output_dir"] / "training_results.csv"

    loss_records = pd.read_csv(loss_path)

    loss_records = loss_records.sort_values(["epoch"]).reset_index(drop=True)

    loss_records["loss_ewm"] = loss_records["combined_loss"].ewm(alpha=0.15).mean()

    epoch_summary = (
        loss_records.groupby("epoch").agg(
            combined_loss_mean=("combined_loss", "mean"),
            combined_loss_std=("combined_loss", "std"),
            mae_mean=("mae", "mean"),
            mse_mean=("mse", "mean")
        ).reset_index()
    )

    results_df = pd.read_csv(results_path)

    results_df["timestamp"] = pd.to_datetime(results_df["timestamp"])

    results_df = results_df.sort_values("epoch").reset_index(drop=True)

    results_df["epoch"] = results_df["epoch"].astype(int)

    results_df["cumulative_time"] = results_df["epoch_time"].cumsum()

    results_df["val_loss_delta"] = results_df["val_loss"].diff()

    results_df["train_val_gap"] = results_df["val_loss"] - results_df["train_loss"]

    results_df["val_mae_delta"] = results_df["val_mae"].diff()

    results_df["epoch_time_rolling"] = results_df["epoch_time"].rolling(5, min_periods=1).mean()

    results_df["memory_headroom_mb"] = results_df["memory_mb"].max() - results_df["memory_mb"]

    merged_metrics = results_df.merge(epoch_summary, on="epoch", how="left")

    merged_metrics["val_loss_rolling"] = merged_metrics["val_loss"].rolling(5, min_periods=1).mean()

    merged_metrics["train_loss_rolling"] = merged_metrics["train_loss"].rolling(5, min_periods=1).mean()

    analytics = {
        "loss_records": loss_records,
        "epoch_summary": epoch_summary,
        "results": results_df,
        "merged_metrics": merged_metrics
    }

    return analytics


def load_scalers(paths: Dict[str, Path]) -> Tuple[Any, Any]:
    """Load cached scalers, regenerating them via training pipeline if missing."""
    scaler_X_path = paths["scaler_X"]

    scaler_y_path = paths["scaler_y"]

    pipeline_ran = False

    def ensure_pipeline() -> None:
        nonlocal pipeline_ran

        if pipeline_ran: return

        complete_data_pipeline(csv_path=str(paths["data_path"]))

        pipeline_ran = True

    try:
        scaler_X = joblib.load(scaler_X_path)
    except FileNotFoundError:
        ensure_pipeline()

        scaler_X = joblib.load(scaler_X_path)

    try:
        scaler_y = joblib.load(scaler_y_path)
    except FileNotFoundError:
        ensure_pipeline()

        scaler_y = joblib.load(scaler_y_path)

    return scaler_X, scaler_y


def load_particle_data(paths: Dict[str, Path]) -> pd.DataFrame:
    """Load particle simulation data with validation safeguards."""
    dataset = load_and_validate_data(csv_path=str(paths["data_path"]))

    if "particle_id" in dataset.columns:
        dataset = dataset.sort_values("particle_id").reset_index(drop=True)
    else:
        dataset = dataset.reset_index(drop=True)

    return dataset

In [ ]:
def build_model_from_config(model_config: Dict[str, Any], training_config: Dict[str, Any]) -> tf.keras.Model:
    """Instantiate a compiled model that mirrors the training setup."""
    config_payload = dict(model_config)

    config_payload.update(training_config)

    config_payload.setdefault("enable_weight_oscillation_dampener", True)

    input_shape = (len(INPUT_FEATURES),)

    output_shape = len(OUTPUT_TARGETS)

    network = AdvancedNeuralNetwork(input_shape=input_shape, output_shape=output_shape, config=config_payload)

    network.compile_model()

    return network.model


def load_model_checkpoint(paths: Dict[str, Path], model_config: Dict[str, Any], training_config: Dict[str, Any], checkpoint_index: pd.DataFrame, checkpoint_name: Optional[str] = None) -> Tuple[Optional[tf.keras.Model], Optional[Dict[str, Any]]]:
    """Load model weights from the selected checkpoint."""
    if checkpoint_index.empty: return None, None

    if checkpoint_name is None:
        selected_row = checkpoint_index.iloc[-1]
    else:
        if checkpoint_name not in checkpoint_index["name"].values: return None, None

        selected_row = checkpoint_index.loc[checkpoint_index["name"] == checkpoint_name].iloc[0]

    weights_path = paths["project_root"] / selected_row["path"]

    tf.keras.backend.clear_session()

    model = build_model_from_config(model_config=model_config, training_config=training_config)

    model.load_weights(weights_path)

    metadata = {
        "epoch": int(selected_row["epoch"]),
        "weights_path": str(weights_path.relative_to(paths["project_root"])),
        "size_bytes": int(selected_row["size_bytes"]),
        "size_readable": selected_row.get("size_readable"),
        "modified": selected_row["modified"],
        "parameter_count": int(model.count_params())
    }

    return model, metadata


def compute_predictions(model: Optional[tf.keras.Model], scaler_X: Any, scaler_y: Any, particle_df: pd.DataFrame, sample_size: int = 256) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """Generate predictions and residual analytics using stored scalers."""
    if model is None: return pd.DataFrame(), {}

    feature_subset = particle_df[INPUT_FEATURES].copy()

    if sample_size and len(feature_subset) > sample_size:
        feature_subset = feature_subset.sample(sample_size, random_state=ANALYSIS_SEED).sort_index()

    scaled_inputs = scaler_X.transform(feature_subset.values) if scaler_X is not None else feature_subset.values

    predictions_scaled = model.predict(scaled_inputs, verbose=0)

    predictions = scaler_y.inverse_transform(predictions_scaled) if scaler_y is not None else predictions_scaled

    actual_outputs = particle_df.loc[feature_subset.index, OUTPUT_TARGETS].values

    residuals = predictions - actual_outputs

    residual_df = pd.DataFrame(index=feature_subset.index)

    if "particle_id" in particle_df.columns:
        residual_df["particle_id"] = particle_df.loc[feature_subset.index, "particle_id"]

    for idx, target in enumerate(OUTPUT_TARGETS):
        residual_df[f"actual_{target}"] = actual_outputs[:, idx]

        residual_df[f"pred_{target}"] = predictions[:, idx]

        residual_df[f"residual_{target}"] = residuals[:, idx]

    residual_df["residual_norm"] = np.linalg.norm(residuals, axis=1)

    residual_norm_mean = residual_df["residual_norm"].mean()

    residual_norm_std = residual_df["residual_norm"].std(ddof=0)

    if residual_norm_std and residual_norm_std > 0:
        residual_df["residual_norm_z"] = (residual_df["residual_norm"] - residual_norm_mean) / residual_norm_std

    mae_value = float(np.mean(np.abs(residuals)))

    rmse_value = float(np.sqrt(np.mean(np.square(residuals))))

    target_metrics: Dict[str, Dict[str, float]] = {}

    for idx, target in enumerate(OUTPUT_TARGETS):
        target_residuals = residuals[:, idx]

        target_metrics[target] = {
            "mae": float(np.mean(np.abs(target_residuals))),
            "rmse": float(np.sqrt(np.mean(np.square(target_residuals)))),
            "bias": float(np.mean(target_residuals))
        }

    metrics: Dict[str, Any] = {
        "samples": int(len(residual_df)),
        "mae": mae_value,
        "rmse": rmse_value,
        "residual_norm_median": float(residual_df["residual_norm"].median()),
        "residual_norm_p95": float(residual_df["residual_norm"].quantile(0.95)),
        "targets": target_metrics
    }

    return residual_df, metrics


def summarize_run_performance(results_df: pd.DataFrame, epoch_summary: pd.DataFrame) -> pd.DataFrame:
    """Create a concise summary of key performance indicators."""
    if results_df.empty: return pd.DataFrame()

    best_epoch_idx = int(results_df["val_loss"].idxmin())

    best_row = results_df.loc[best_epoch_idx]

    final_row = results_df.iloc[-1]

    early_row = results_df.iloc[0]

    improvement = float(early_row["val_loss"] - best_row["val_loss"])

    consistency = float(epoch_summary["combined_loss_std"].tail(5).mean()) if not epoch_summary.empty else float("nan")

    best_r2_row = results_df.loc[results_df["r2_score"].idxmax()]

    summary = pd.DataFrame([
        {"metric": "Best validation loss", "value": best_row["val_loss"], "notes": f"Epoch {int(best_row['epoch'])}"},
        {"metric": "Final validation loss", "value": final_row["val_loss"], "notes": f"Train gap {final_row['train_val_gap']:.4f}"},
        {"metric": "Validation improvement", "value": improvement, "notes": "Drop from first to best epoch"},
        {"metric": "Validation stability (std last 5 epochs)", "value": consistency, "notes": "Lower is more stable"},
        {"metric": "Average epoch time (last 10 epochs)", "value": results_df["epoch_time"].tail(10).mean(), "notes": "Supports batch-size experiments"},
        {"metric": "Peak R²", "value": best_r2_row["r2_score"], "notes": f"Epoch {int(best_r2_row['epoch'])}"},
        {"metric": "Total recorded training time", "value": results_df["epoch_time"].sum(), "notes": "seconds"}
    ])

    return summary


def suggest_hyperparameters(model_config: Dict[str, Any], training_config: Dict[str, Any], config_history: pd.DataFrame, results_df: pd.DataFrame) -> pd.DataFrame:
    """Derive hyperparameter sweep recommendations from observed metrics."""
    if results_df.empty: return pd.DataFrame()

    suggestions: List[Dict[str, Any]] = []

    base_lr = float(model_config.get("learning_rate", 0.001))

    final_window = results_df.tail(5)

    val_loss_range = float(final_window["val_loss"].max() - final_window["val_loss"].min())

    best_epoch = int(results_df.loc[results_df["val_loss"].idxmin(), "epoch"])

    final_epoch = int(results_df.iloc[-1]["epoch"])

    total_epochs = int(training_config.get("epochs", final_epoch + 1))

    if val_loss_range < 0.01 and final_epoch - best_epoch > 5:
        proposals = sorted({round(base_lr * factor, 6) for factor in (0.5, 0.8, 1.2)})

        suggestions.append({
            "parameter": "learning_rate",
            "proposed_values": proposals,
            "rationale": "Validation loss plateaued across the last epochs; nudge the optimizer step to reintroduce progress.",
            "constraints": "Keep BinaryWeightConstraintMax(max_binary_digits=5) engaged for stability."
        })

    train_val_gap = float(final_window["train_val_gap"].mean())

    if train_val_gap > 0.05:
        suggestions.append({
            "parameter": "dropout_rate",
            "proposed_values": [0.05, 0.1, 0.15],
            "rationale": "Consistent validation > training loss points to mild overfitting; mild dropout can regularize activations.",
            "constraints": "Retain enable_weight_oscillation_dampener=True to temper weight swings."
        })

    avg_epoch_time = float(results_df["epoch_time"].tail(10).mean())

    memory_headroom = float(results_df["memory_headroom_mb"].tail(10).mean())

    if avg_epoch_time < 1.5 and memory_headroom > 0:
        baseline_batch = int(training_config.get("batch_size", 16))

        candidate_batches = sorted({baseline_batch, 24, 32})

        suggestions.append({
            "parameter": "batch_size",
            "proposed_values": candidate_batches,
            "rationale": "Epoch time and memory logs show headroom; larger batches could reduce gradient variance.",
            "constraints": "Validate GPU memory against peak usage before committing."
        })

    if final_epoch >= total_epochs - 2:
        extension_epochs = sorted({total_epochs + 10, total_epochs + 20})

        suggestions.append({
            "parameter": "epochs",
            "proposed_values": extension_epochs,
            "rationale": "Best epoch occurs near training ceiling; extending training may unlock additional gains.",
            "constraints": "Monitor for overfitting; stop early if val loss degrades."
        })

    if not config_history.empty and "learning_rate" in config_history.columns:
        grouped = config_history.groupby("learning_rate")["final_r2"].mean().sort_values()

        if len(grouped) > 1:
            top_lr = grouped.idxmax()

            if abs(top_lr - base_lr) / base_lr > 0.2:
                suggestions.append({
                    "parameter": "learning_rate",
                    "proposed_values": [round(float(top_lr), 6)],
                    "rationale": "Historical sweep points to a different learning rate yielding higher final R².",
                    "constraints": "Pair with BinaryWeightConstraintChanges() to keep update granularity consistent."
                })

    if suggestions:
        recommendations = pd.DataFrame(suggestions)

        return recommendations.drop_duplicates(subset=["parameter", "rationale"])

    return pd.DataFrame()

In [ ]:
paths = resolve_project_paths()

display(Markdown(f"**Project root:** `{paths['project_root']}`"))

artifact_status = validate_required_artifacts(paths)

display(Markdown("### Artifact Inventory"))

display(artifact_status)

missing_artifacts = artifact_status.loc[~artifact_status["exists"]]

if not missing_artifacts.empty:
    display(Markdown("⚠️ **Missing artifacts detected. Review notes before continuing.**"))

    display(missing_artifacts)
else:
    display(Markdown("✅ All critical artifacts are present."))

In [ ]:
model_config, training_config, config_history = load_configs(paths)

display(Markdown("### Active Model Configuration"))

display(pd.Series(model_config, name="model_config"))

display(Markdown("### Active Training Configuration"))

display(pd.Series(training_config, name="training_config"))

if not config_history.empty:
    display(Markdown("### Historical Configuration Snapshots"))

    history_columns = [
        col
        for col in [
            "timestamp", "config_id", "learning_rate", "dropout_rate", "train_batch_size", "train_epochs", "best_r2", "final_r2", "r2_delta", "avg_epoch_time_logged", "avg_epoch_time_calc", "total_training_time"
        ]
        if col in config_history.columns
    ]

    display(config_history[history_columns])

    numeric_cols = [col for col in history_columns if config_history[col].dtype.kind in "if"]

    if numeric_cols:
        history_stats = config_history[numeric_cols].describe().transpose()

        display(Markdown("#### Configuration Summary Statistics"))

        display(history_stats)

In [ ]:
analytics = load_training_logs(paths)

loss_records = analytics["loss_records"]

epoch_summary = analytics["epoch_summary"]

results_df = analytics["results"]

merged_metrics = analytics["merged_metrics"]

display(Markdown("### Epoch-Level Performance Summary"))

display(results_df.tail(10)[["epoch", "train_loss", "val_loss", "train_val_gap", "val_loss_delta", "epoch_time"]])

performance_snapshot = summarize_run_performance(results_df, epoch_summary)

display(Markdown("### Key Performance Indicators"))

display(performance_snapshot)

display(Markdown("#### Loss Distribution by Epoch"))

display(epoch_summary.tail(10))

display(Markdown("#### Exponential Moving Average of Combined Loss"))

display(loss_records.tail(10)[["epoch", "combined_loss", "loss_ewm"]])

In [ ]:
particle_df = load_particle_data(paths)

scaler_X, scaler_y = load_scalers(paths)

display(Markdown("### Particle Data Snapshot"))

display(Markdown(f"Dataset shape: **{particle_df.shape[0]}** rows × **{particle_df.shape[1]}** columns"))

display(particle_df.head())

display(Markdown("#### Descriptive Statistics"))

display(particle_df.describe(include="all").transpose())

missing_counts = particle_df.isna().sum()

if missing_counts.any():
    display(Markdown("#### Missing Value Audit"))

    display(missing_counts[missing_counts > 0])

In [ ]:
checkpoint_index = list_checkpoint_weights(paths)

display(Markdown("### Available Weight Checkpoints"))

if checkpoint_index.empty:
    display(Markdown("No checkpoints found. Run training to generate weight artifacts."))
else:
    display(checkpoint_index)

model, checkpoint_meta = load_model_checkpoint(paths, model_config, training_config, checkpoint_index)

if checkpoint_meta is not None:
    display(Markdown(f"Loaded checkpoint: **epoch {checkpoint_meta['epoch']}** from `{checkpoint_meta['weights_path']}`"))

    display(pd.Series(checkpoint_meta))

In [ ]:
residuals_df, residual_metrics = compute_predictions(model, scaler_X, scaler_y, particle_df)

if residual_metrics:
    display(Markdown("### Residual Metrics"))

    overall_metrics = {key: value for key, value in residual_metrics.items() if key != "targets"}

    display(pd.Series(overall_metrics, name="residual_metrics"))

    target_metrics = pd.DataFrame(residual_metrics["targets"]).transpose()

    display(Markdown("#### Per-Target Residual Summary"))

    display(target_metrics)

if not residuals_df.empty:
    display(Markdown("### Residual Sample"))

    display(residuals_df.head())

In [ ]:
figures_dir = paths["figures_dir"]

# Loss trend
fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data=results_df, x="epoch", y="train_loss", ax=ax, label="Train Loss")
sns.lineplot(data=results_df, x="epoch", y="val_loss", ax=ax, label="Validation Loss")
val_std = results_df["val_loss"].rolling(5, min_periods=1).std()
ax.fill_between(results_df["epoch"], results_df["val_loss"] - val_std, results_df["val_loss"] + val_std, color="tab:blue", alpha=0.1)
ax.set_title("Training vs Validation Loss")
ax.set_ylabel("Loss")
fig.tight_layout()
loss_curve_path = figures_dir / "loss_curves.png"
fig.savefig(loss_curve_path, dpi=200)
plt.close(fig)
display(Markdown(f"Saved loss curves to `{loss_curve_path}`"))

# Train vs validation gap
fig, ax = plt.subplots(figsize=(10, 4))
sns.lineplot(data=results_df, x="epoch", y="train_val_gap", ax=ax, color="tab:red")
ax.axhline(0, linestyle="--", color="grey", linewidth=1)
ax.set_title("Train vs Validation Gap")
ax.set_ylabel("Val - Train Loss")
fig.tight_layout()
gap_plot_path = figures_dir / "train_val_gap.png"
fig.savefig(gap_plot_path, dpi=200)
plt.close(fig)
display(Markdown(f"Saved train/val gap chart to `{gap_plot_path}`"))

# Learning rate vs final loss metrics
if not config_history.empty:
    lr_df = config_history.copy()

    fig, ax = plt.subplots(figsize=(8, 5))
    sns.scatterplot(data=lr_df, x="learning_rate", y="final_r2", size="total_training_time", hue="final_r2", palette="viridis", ax=ax)
    ax.set_title("Learning Rate vs Final R²")
    ax.set_xlabel("Learning Rate")
    ax.set_ylabel("Final R²")
    fig.tight_layout()
    lr_plot_path = figures_dir / "learning_rate_vs_r2.png"
    fig.savefig(lr_plot_path, dpi=200)
    plt.close(fig)
    display(Markdown(f"Saved learning-rate diagnostics to `{lr_plot_path}`"))

# Residual histogram
if not residuals_df.empty:
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.histplot(residuals_df["residual_norm"], bins=30, ax=ax, kde=True, color="tab:orange")
    ax.set_title("Residual Norm Distribution")
    ax.set_xlabel("Residual Norm")
    fig.tight_layout()
    residual_hist_path = figures_dir / "residual_norm_hist.png"
    fig.savefig(residual_hist_path, dpi=200)
    plt.close(fig)
    display(Markdown(f"Saved residual histogram to `{residual_hist_path}`"))

    residual_columns = [col for col in residuals_df.columns if col.startswith("residual_") and any(col.endswith(target) for target in OUTPUT_TARGETS)]

    if residual_columns:
        fig, ax = plt.subplots(figsize=(10, 5))
        melted = residuals_df[residual_columns].melt(var_name="target", value_name="residual")
        sns.boxplot(data=melted, x="target", y="residual", ax=ax)
        ax.tick_params(axis="x", rotation=45)
        ax.set_title("Residual Distribution by Target")
        fig.tight_layout()
        residual_box_path = figures_dir / "residual_distribution_by_target.png"
        fig.savefig(residual_box_path, dpi=200)
        plt.close(fig)
        display(Markdown(f"Saved residual distribution boxplot to `{residual_box_path}`"))

# Correlation heatmap
heatmap_features = ["train_loss", "val_loss", "train_mae", "val_mae", "r2_score", "epoch_time", "train_val_gap", "memory_headroom_mb"]
usable_cols = [col for col in heatmap_features if col in merged_metrics.columns]

if usable_cols:
    corr_matrix = merged_metrics[usable_cols].corr()

    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", ax=ax)
    ax.set_title("Metric Correlation Heatmap")
    fig.tight_layout()
    heatmap_path = figures_dir / "metric_correlation_heatmap.png"
    fig.savefig(heatmap_path, dpi=200)
    plt.close(fig)
    display(Markdown(f"Saved correlation heatmap to `{heatmap_path}`"))

In [ ]:
recommendations_df = suggest_hyperparameters(model_config, training_config, config_history, results_df)

if not recommendations_df.empty:
    display(Markdown("### Recommended Hyperparameter Sweeps"))

    display(recommendations_df)

else:
    display(Markdown("No immediate hyperparameter adjustments detected beyond current configuration."))

In [ ]:
insight_items: List[str] = []

if not results_df.empty:
    final_row = results_df.iloc[-1]

    best_row = results_df.loc[results_df["val_loss"].idxmin()]

    insight_items.append(f"Best validation loss {best_row['val_loss']:.4f} at epoch {int(best_row['epoch'])}.")

    insight_items.append(f"Validation plateau range over last window: {(results_df.tail(5)['val_loss'].max() - results_df.tail(5)['val_loss'].min()):.4f}.")

    insight_items.append(f"Train/val gap at final epoch: {final_row['train_val_gap']:.4f}.")

    insight_items.append(f"Cumulative training time logged: {results_df['epoch_time'].sum():.1f} seconds.")

if residual_metrics:
    insight_items.append(f"Mean absolute residual across sampled predictions: {residual_metrics['mae']:.4f}.")

    insight_items.append(f"95th percentile residual norm: {residual_metrics['residual_norm_p95']:.4f}.")

if not recommendations_df.empty:
    suggested = ", ".join(recommendations_df["parameter"].unique())

    insight_items.append(f"Hyperparameter sweep targets: {suggested}.")

missing_artifacts = artifact_status.loc[~artifact_status["exists"] & artifact_status["critical"]]

if not missing_artifacts.empty:
    missing_list = ", ".join(missing_artifacts["artifact"].tolist())

    insight_items.append(f"Critical artifacts missing: {missing_list}.")

if not insight_items:
    insight_items.append("Insufficient data to derive insights.")

display(Markdown("### Insight Summary"))

for item in insight_items:
    display(Markdown(f"- {item}"))

In [ ]:
def run_notebook_smoke_test() -> Dict[str, Any]:
    """Validate that core notebook stages complete without exceptions."""
    status = {
        "critical_artifacts_present": bool(artifact_status.loc[artifact_status["critical"] & ~artifact_status["exists"]].empty),
        "config_history_entries": int(len(config_history)),
        "loss_records": int(len(loss_records)),
        "results_records": int(len(results_df)),
        "residual_samples": int(len(residuals_df)),
        "recommendations": int(len(recommendations_df)),
        "figures_exported": len(list(paths["figures_dir"].glob("*.png"))),
        "latest_checkpoint_epoch": int(checkpoint_meta["epoch"]) if checkpoint_meta else None
    }

    return status


smoke_test_status = run_notebook_smoke_test()

display(Markdown("### Validation Checklist"))

display(pd.Series(smoke_test_status, name="notebook_validation"))

## Actionable Next Steps

- Re-run the training pipeline after trialing the proposed learning-rate, dropout, and batch-size combinations; capture new config snapshots for comparison.
- Promote saved figures under `training_output/analysis/figures/` into experiment reports or dashboards.
- Extend this notebook with automated sweeps (GridSearch or Bayesian optimization) once additional configuration diversity is available.

### Reuse Tips

- Parameterize `sample_size` within `compute_predictions` to scale residual analysis for larger datasets.
- Import this notebook’s helper functions via `%run experiment_analysis_framework.ipynb` inside future analysis notebooks for rapid setup.
- Store additional diagnostics (e.g., feature importance, SHAP values) within the `analysis` directory for cross-experiment benchmarking.